```{contents}
```
## Static Edge

A **Static Edge** in LangGraph is a **deterministic, unconditional transition** between two nodes in the execution graph.
It defines a **fixed control flow path** that is always followed once the source node completes.

---

### **1. Intuition**

A static edge behaves like a **hard-wired wire** in a circuit:

> *When node A finishes, execution always moves to node B.*

There is **no decision**, **no branching**, and **no runtime evaluation**.

This makes static edges ideal for modeling **pipeline-style workflows**.

---

### **2. Formal Definition**

A static edge is created using:

```python
builder.add_edge("A", "B")
```

Semantics:

```
Execute A  →  then always execute B
```

---

### **3. Execution Model**

```
[ Node A ] ───▶ [ Node B ] ───▶ [ Node C ]
```

Execution order is **fully determined at compile time**.

No state inspection occurs at the edge.

---

### **4. Why Static Edges Matter**

| Property      | Value               |
| ------------- | ------------------- |
| Determinism   | Fully predictable   |
| Performance   | No routing overhead |
| Debuggability | Simple trace        |
| Safety        | No unexpected paths |
| Optimizable   | Compiler-friendly   |

---

### **5. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class State(TypedDict):
    text: str

def preprocess(state):
    return {"text": state["text"].lower()}

def analyze(state):
    return {"text": state["text"] + " analyzed"}

builder = StateGraph(State)
builder.add_node("pre", preprocess)
builder.add_node("analyze", analyze)

builder.set_entry_point("pre")
builder.add_edge("pre", "analyze")
builder.add_edge("analyze", END)

graph = builder.compile()

print(graph.invoke({"text": "HELLO"}))
```

Output:

```
{'text': 'hello analyzed'}
```

---

### **6. Static Edge vs Conditional Edge**

| Feature             | Static Edge | Conditional Edge |
| ------------------- | ----------- | ---------------- |
| Decision at runtime | ❌           | ✅                |
| Branching           | ❌           | ✅                |
| Overhead            | Minimal     | Higher           |
| Predictability      | Maximum     | Depends on state |
| Use case            | Pipelines   | Logic & routing  |

---

### **7. Typical Use Cases**

* ETL pipelines
* Prompt preprocessing
* LLM → tool → postprocessing
* Validation pipelines
* Deterministic business workflows

---

### **8. Combining Static & Cyclic Edges**

Static edges are often embedded inside loops:

```
Reason ─▶ Act ─▶ Observe
  ▲                 │
  └──── Static ─────┘
```

Only the **loop decision** is conditional; the steps inside are static.

---

### **9. Production Design Guidelines**

* Use static edges for **core pipeline flow**
* Add conditional edges only for **routing decisions**
* Prefer static edges for **performance-critical paths**
* Keep static pipelines **short and composable**

---

### **10. Mental Model**

> **Static edges = compile-time wiring**
> **Conditional edges = runtime decisions**

LangGraph combines both to express **highly reliable execution systems**.


### Demonstration

In [1]:
from typing import TypedDict

class State(TypedDict):
    text: str

def preprocess(state: State):
    print("Preprocess running")
    return {"text": state["text"].lower()}

def analyze(state: State):
    print("Analyze running")
    return {"text": state["text"] + " | analyzed"}

def postprocess(state: State):
    print("Postprocess running")
    return {"text": state["text"] + " | ready"}


In [2]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("pre", preprocess)
builder.add_node("analyze", analyze)
builder.add_node("post", postprocess)

builder.set_entry_point("pre")

builder.add_edge("pre", "analyze")
builder.add_edge("analyze", "post")
builder.add_edge("post", END)

graph = builder.compile()


In [3]:
result = graph.invoke({"text": "HELLO WORLD"})
print(result)


Preprocess running
Analyze running
Postprocess running
{'text': 'hello world | analyzed | ready'}
